## Modules à importer

In [1]:
import math
import scipy
import pickle
import numpy as np
import pandas as pd
import iminuit as im
import ipywidgets as ipw
from importlib import reload
from scipy.stats import poisson
from variaIa import baserate
from variaIa import stretchevol
from astropy.cosmology import Planck15 as cdata
from IPython.display import display, Latex
import matplotlib.pyplot as plt
import random

## Définitions pandas, surveys, $z[:z_{\text{max}}]$, $x_1[:z_{\text{max}}]$, mean, std et données totales

In [2]:
cons = ipw.Checkbox(
    value=False,
    description='Conservative')
display(cons)

Checkbox(value=False, description='Conservative')

In [36]:
d = pd.read_csv('../Data/data_cheat.csv', sep=' ', index_col = 'CID')
d_snf = pd.read_csv('../Data/lssfr_paper_full_sntable.csv',sep=',')

surveys = ['SNF', 'SDSS', 'PS1', 'SNLS', 'HST']

#if cons.value:
surv = {'SNF':  d_snf, # .loc[d_snf['name'].str.contains('SNF|LSQ|PTF',na=False,regex=True)],
        'SDSS': d[d['IDSURVEY'] == 1],
        'PS1':  d[d['IDSURVEY'] == 15],
        'SNLS': d[d['IDSURVEY'] == 4],
        'HST':  d[d['IDSURVEY'].isin([101, 100, 106])]}
#else:
#    surv = {'SNF':  d_snf,
#            'SDSS': d[d['IDSURVEY'] == 1],
#            'PS1':  d[d['IDSURVEY'] == 15],
#            'SNLS': d[d['IDSURVEY'] == 4],
#            'HST':  d[d['IDSURVEY'].isin([101, 100, 106])]}

with open('../Data/zmax_mlim', 'rb') as f:
    z_max = pickle.load(f)

z_max['SNF'] = [10, 10]
z_max['HST'] = [10, 10]

dgmap = plt.cm.get_cmap('viridis')
colors = {'SNF': dgmap(0),
          'SDSS': dgmap(50),
          'PS1': dgmap(125),
          'SNLS': dgmap(200),
          'HST': dgmap(300)}

zmax_cuts = dict()
z_zcuts = dict()
x1_zcuts = dict()
x1_err_zcuts = dict()

if cons.value:
    for survey in surveys[1:]:
        zmax_cuts[survey] = np.where(surv[survey].zCMB.values < z_max[survey][0])
        z_zcuts[survey] = surv[survey].zCMB.values[zmax_cuts[survey]]
        x1_zcuts[survey] = surv[survey].x1.values[zmax_cuts[survey]]
        x1_err_zcuts[survey] = surv[survey].x1ERR.values[zmax_cuts[survey]]

        zmax_cuts['SNF'] = np.where(surv['SNF']['host.zcmb'].values < z_max['SNF'][0])
        z_zcuts['SNF'] = surv['SNF']['host.zcmb'].values[zmax_cuts['SNF']]
        x1_zcuts['SNF'] = surv['SNF']['salt2.X1'].values[zmax_cuts['SNF']]
        x1_err_zcuts['SNF'] = surv['SNF']['salt2.X1.err'].values[zmax_cuts['SNF']]
else:
    for survey in surveys[1:]:
        zmax_cuts[survey] = np.where(surv[survey].zCMB.values < z_max[survey][-1])
        z_zcuts[survey] = surv[survey].zCMB.values[zmax_cuts[survey]]
        x1_zcuts[survey] = surv[survey].x1.values[zmax_cuts[survey]]
        x1_err_zcuts[survey] = surv[survey].x1ERR.values[zmax_cuts[survey]]

        zmax_cuts['SNF'] = np.where(surv['SNF']['host.zcmb'].values < z_max['SNF'][-1])
        z_zcuts['SNF'] = surv['SNF']['host.zcmb'].values[zmax_cuts['SNF']]
        x1_zcuts['SNF'] = surv['SNF']['salt2.X1'].values[zmax_cuts['SNF']]
        x1_err_zcuts['SNF'] = surv['SNF']['salt2.X1.err'].values[zmax_cuts['SNF']]

datax_all = np.concatenate(
    (np.concatenate(
        (np.concatenate(
            (np.concatenate((x1_zcuts['SNF'],
                             x1_zcuts['SDSS'])),
             x1_zcuts['PS1'])),
         x1_zcuts['SNLS'])),
     x1_zcuts['HST']))
                                                     
datax_err_all = np.concatenate(
    (np.concatenate(
        (np.concatenate(
            (np.concatenate((x1_err_zcuts['SNF'],
                             x1_err_zcuts['SDSS'])),
             x1_err_zcuts['PS1'])),
         x1_err_zcuts['SNLS'])),
     x1_err_zcuts['HST']))
                                                     
dataz_all = np.concatenate(
    (np.concatenate(
        (np.concatenate(
            (np.concatenate((z_zcuts['SNF'],
                             z_zcuts['SDSS'])),
             z_zcuts['PS1'])),
         z_zcuts['SNLS'])),
     z_zcuts['HST']))

z_mean = dict()
z_std = dict()
x1_mean = dict()
x1_std = dict()

for survey in surveys:
    z_mean[survey] = np.mean(z_zcuts[survey])
    z_std[survey] = np.std(z_zcuts[survey])
    x1_mean[survey] = np.mean(x1_zcuts[survey])
    x1_std[survey] = np.std(x1_zcuts[survey])

## Set print level

In [37]:
p_l = ipw.IntSlider(min = 0, max = 1, value = 0, description='print_level:')
display(p_l)

IntSlider(value=0, description='print_level:', max=1)

## Minimisation des modèles à mettre dans le _pandas_

In [38]:
reload(stretchevol)

########################################### Evol Simple

evol1G1M1S = stretchevol.Evol1G1M1S()
evol1G1M1S.set_data(dataz_all, datax_all, datax_err_all)

print('1G1M1S')

evol1G1M1S.minimize(p_l.value)

########################################### Evol Kessler

evol1G1M2S = stretchevol.Evol1G1M2S()
evol1G1M2S.set_data(dataz_all, datax_all, datax_err_all)

print('1G1M2S')

evol1G1M2S.minimize(p_l.value)

# evol1G1M2S.traceur()

########################################### Evol Howell

evol2G2M2S = stretchevol.Evol2G2M2S()
evol2G2M2S.set_data(dataz_all, datax_all, datax_err_all)

print('2G2M2S')

evol2G2M2S.minimize(p_l.value)
#evol2G2M2S.plotter()

########################################### Evol Howell Fixed

evol2G2M2SF = stretchevol.Evol2G2M2SF()
evol2G2M2SF.set_data(dataz_all, datax_all, datax_err_all)

print('2G2M2SF')

evol2G2M2SF.minimize(p_l.value)

########################################### Evol NR 1S

evol3G2M1S = stretchevol.Evol3G2M1S()
evol3G2M1S.set_data(dataz_all, datax_all, datax_err_all)

print('3G2M1S')

evol3G2M1S.minimize(p_l.value)

########################################### Evol NR 1S Fixed

evol3G2M1SF = stretchevol.Evol3G2M1SF()
evol3G2M1SF.set_data(dataz_all, datax_all, datax_err_all)

print('3G2M1SF')

evol3G2M1SF.minimize(p_l.value, limit_f = (0,1), limit_a = (0,1))

########################################### Evol NR 1S SNF

evol3G2M1SSNF = stretchevol.Evol3G2M1SSNF()
evol3G2M1SSNF.set_lssfr(d_snf['salt2.X1'], d_snf['salt2.X1.err'],
                        d_snf['lssfr'], d_snf['lssfr.err_down'], d_snf['lssfr.err_up'],
                        d_snf['p(prompt)'])

evol3G2M1SSNF.minimize(p_l.value)

# evol3G2M1SSNF.plotter("3G2M1SSNF")

########################################### Evol NR 2S

evol3G2M2S = stretchevol.Evol3G2M2S()
evol3G2M2S.set_data(dataz_all, datax_all, datax_err_all)

print('3G2M2S')

evol3G2M2S.minimize(p_l.value)

########################################### Evol NR 2S Fixed

evol3G2M2SF = stretchevol.Evol3G2M2SF()
evol3G2M2SF.set_data(dataz_all, datax_all, datax_err_all)

print('3G2M2SF')

evol3G2M2SF.minimize(p_l.value, limit_f = (0,1))

########################################### Evol NR 2S SNF

evol3G2M2SSNF = stretchevol.Evol3G2M2SSNF()
evol3G2M2SSNF.set_lssfr(d_snf['salt2.X1'], d_snf['salt2.X1.err'],
                        d_snf['lssfr'], d_snf['lssfr.err_down'], d_snf['lssfr.err_up'],
                        d_snf['p(prompt)'])

print('3G2M2SSNF')

evol3G2M2SSNF.minimize(p_l.value)

# evol3G2M2SSNF.plotter()

########################################### Evol NR 3S

evol3G3M3S = stretchevol.Evol3G3M3S()
evol3G3M3S.set_data(dataz_all, datax_all, datax_err_all)

print('3G3M3S')

evol3G3M3S.minimize(p_l.value)

########################################### Evol NR 3S Fixed

evol3G3M3SF = stretchevol.Evol3G3M3SF()
evol3G3M3SF.set_data(dataz_all, datax_all, datax_err_all)

print('3G3M3SF')

evol3G3M3SF.minimize(0, limit_f = (0,1), limit_a = (0,1))
#evol3G3M3SF.migrad_out[0]

#fig=plt.gcf()
#fig.savefig('/home/nora/Documents/Thèse/Images/3G2M2SSNF.pdf', bbox_inches='tight')

1G1M1S
1G1M2S
2G2M2S
2G2M2SF
3G2M1S
3G2M1SF
3G2M2S
3G2M2SF
3G2M2SSNF
3G3M3S
3G3M3SF


## Get proba

In [39]:
def get_proba(model):
    return np.exp((evol3G2M1S.get_aicc() - model.get_aicc())/2)

In [40]:
modèles = [evol3G2M1S, evol3G2M2S, evol2G2M2S, evol3G3M3S,
           evol2G2M2SF, evol3G2M2SF, evol3G3M3SF,
           evol1G1M2S, evol1G1M1S, evol3G2M1SF ]

d_mod_comp = pd.DataFrame({'Name': [k.__doc__ for k in modèles],
                           'Fixed': [k.FIXED for k in modèles],
                           'Free param': [len(k.FREEPARAMETERS) for k in modèles],
                           'ln $\mathcal{L}$': [k.get_logl() for k in modèles],
                           'AICc': [k.get_aicc() for k in modèles],
                           '$\Delta$ AICc': [evol3G2M1S.get_aicc() - k.get_aicc() for k in modèles],
                           'Proba': [get_proba(k) for k in modèles]})

d_mod_comp

,Name,Fixed,Free param,ln $\mathcal{L}$,AICc,$\Delta$ AICc,Proba
0,Base$-(\sigma_2)$,False,4,1194.543999,1202.634089,0.000000,1.000000e+00
1,Base,False,5,1194.210215,1204.345655,-1.711565,4.249504e-01
2,Howell,False,4,1195.272704,1203.362794,-0.728705,6.946464e-01
3,"Base$+(\mu_1^{\text{O}}, \sigma_1^{\text{O}})$",False,7,1189.561099,1203.815067,-1.180978,5.540563e-01
4,Howell,True,5,1199.110590,1209.246030,-6.611941,3.666362e-02
5,Base,True,6,1199.110544,1211.300590,-8.666500,1.312482e-02
6,"Base$+(\mu_1^{\text{O}}, \sigma_1^{\text{O}})$",True,8,1190.741652,1207.068924,-4.434835,1.088900e-01
7,Assymetric,True,3,1304.214416,1310.268349,-107.634259,4.241482e-24
8,Gaussian,True,2,1246.769024,1250.795930,-48.161840,3.481683e-11
9,Base$-(\sigma_2)$,True,5,1200.772216,1210.907657,-8.273567,1.597415e-02


In [13]:
path = '../Data/df_comparaison'
if cons.value:
    path += '_cons'
path += '.dat'
d_mod_comp.to_csv(path)